In [1]:
import sys
print(sys.executable)
print(sys.path)

c:\Users\N1259534\AppData\Local\anaconda3\envs\qaoa_sim\python.exe
['c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\envs\\qaoa_sim\\python313.zip', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\envs\\qaoa_sim\\DLLs', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\envs\\qaoa_sim\\Lib', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\envs\\qaoa_sim', '', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\envs\\qaoa_sim\\Lib\\site-packages', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\envs\\qaoa_sim\\Lib\\site-packages\\win32', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\envs\\qaoa_sim\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\envs\\qaoa_sim\\Lib\\site-packages\\Pythonwin']


In [2]:

import numpy as np
from itertools import combinations
import scipy
import networkx as nx
import matplotlib.pyplot as plt

from QAOA_objective import get_qaoa_objective


ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.

In [3]:
import numba.cuda
print(numba.cuda.is_available())

False


In [4]:
import cupy as cp

c:\Users\N1259534\AppData\Local\anaconda3\envs\qaoa_sim\Lib\site-packages\cupy\_environment.py:541: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [ ]:
#Step 1: Generate terms

N = 10
np.random.seed(1)
terms = [(np.random.normal(), spin_pair) for spin_pair in combinations(range(N), r=2)]
print(terms)

In [ ]:
G = nx.Graph()
for weight, (node1, node2) in terms:
    G.add_edge(node1, node2, wight=round(weight, 2))
    
plt.figure(figsize=(3, 1.5))
pos = nx.spring_layout(G)  # Layout for visualization
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=500, font_size=12)


plt.show()

In [ ]:
# Step 2: get QAOA objective
# Get objective
p =4
f = get_qaoa_objective(N, terms=terms, parameterization='theta', simulator='gpu')

In [ ]:
# Step 3: Optimize QAOA parameters
p = 4
initial_gamma = -1*np.linspace(0, 1, p)
initial_beta = np.linspace(1, 0, p)

res = scipy.optimize.minimize(f, np.hstack([initial_gamma, initial_beta]), method='COBYLA', options={'rhobeg': 0.01})

In [ ]:
nvcc --version

In [ ]:
print(f"Expected QAOA solution quality: {res.fun:.5f}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(-res.x[:p], label=r"$\gamma$")
plt.plot(res.x[p:], label=r"$\beta$")
plt.title('Optimized QAOA parameters')
plt.legend()

In [ ]:
def f_from_terms_ground_truth(s):
    """ground truth for debugging
    """
    out = 0
    for coeff, (i, j) in terms:
        out += coeff * s[i] * s[j]
    return out

In [ ]:
print(f"True minimum: {brute_force(f_from_terms_ground_truth, N, minimize=True)[0]:.5f}")
